In [ ]:
%run reader.ipynb

In [ ]:
airs_data_asc = airs_read('AIRS/AIRS3STD_7_0_TotH2OVap_A.20160701-20220531.68W_76N_68W_76N.csv')
airs_data_disc = airs_read('AIRS/AIRS3STD_7_0_TotH2OVap_D.20160701-20220531.68W_76N_68W_76N.csv')
cimel_data = cimel_read('CIMEL/20170101_20211231_Thule.lev20')
ftir_data = ftir_read('FTIR/h2o_FTIR_2016_2020.dat')
mod_data = modis_read('MODIS/MOD05_5km_totale.csv')
myd_data = modis_read('MODIS/MYD05_5km_totale.csv')
radiosounding_data = radiosounding_read('RADIOSOUNDING/PWVradiosondeThule.txt')
vespa_data = vespa_read('VESPA/Vapor_20160716_20220311.txt')

In [ ]:
year=[2016,2017,2018,2019]
gnss_data = pd.DataFrame()
hatpro_data = pd.DataFrame()

for year in year:
    gnss_data = gnss_data.append(gnss_read(
        f'GNSS/THU3_{year}global.txt',year))
    hatpro_data = hatpro_data.append(hatpro_read(
        f'HATPRO/IWV_{year}_HATPRO.txt',year))

### Drop rows with water values > 50mm 

In [ ]:
mod_data = df_drop(mod_data, mod_data.IWV_MODIS > 50.)
myd_data = df_drop(myd_data, myd_data.IWV_MODIS > 50. )

### Drop rows with distance values > 10 km 

In [ ]:
mod_data = distance_from_station(mod_data,drop_above=10.)
myd_data = distance_from_station(myd_data,drop_above=10.)

### Subset dataset less than 10 minutes

In [ ]:
new_myd_vespa = closest_in_time2(myd_data,vespa_data,'IWV_MODIS','IWV_THAAO',on='Time',delta_time='10m')
new_mod_vespa = closest_in_time2(mod_data,vespa_data,'IWV_MODIS','IWV_THAAO',on='Time',delta_time='10m')

### Subset AIRS dataset less than 24 hours

In [ ]:
#AIRS è su AQUA quindi confronto solo con myd

new_mod_airs_asc = closest_in_time2(mod_data,airs_data_asc,'IWV_MODIS','IWV[mm]',on='Time',delta_time='24h')
new_myd_airs_asc = closest_in_time2(myd_data,airs_data_asc,'IWV_MODIS','IWV[mm]',on='Time',delta_time='24h')
new_mod_airs_disc = closest_in_time2(mod_data,airs_data_disc,'IWV_MODIS','IWV[mm]',on='Time',delta_time='24h')
new_myd_airs_disc = closest_in_time2(myd_data,airs_data_disc,'IWV_MODIS','IWV[mm]',on='Time',delta_time='24h')

In [ ]:
new_mod_airs_asc = new_mod_airs_asc.dropna(subset=['IWV_MODIS','IWV[mm]'])
new_myd_airs_asc = new_myd_airs_asc.dropna(subset=['IWV_MODIS','IWV[mm]'])
new_mod_airs_disc = new_mod_airs_disc.dropna(subset=['IWV_MODIS','IWV[mm]'])
new_myd_airs_disc = new_myd_airs_disc.dropna(subset=['IWV_MODIS','IWV[mm]'])

new_modis_TOT_airs_asc = pd.concat([new_mod_airs_asc,new_myd_airs_asc], sort=True)
new_modis_TOT_airs_disc = pd.concat([new_mod_airs_disc,new_myd_airs_disc], sort=True)

new_modis_TOT_airs = pd.concat([new_modis_TOT_airs_asc,new_modis_TOT_airs_disc], sort=True)

%store new_mod_airs_asc new_mod_airs_disc new_modis_TOT_airs new_modis_TOT_airs_asc new_modis_TOT_airs_disc
%store new_myd_airs_asc new_myd_airs_disc

In [ ]:
new_mod_vespa = new_mod_vespa.dropna(subset=['IWV_MODIS','IWV_THAAO'])
new_myd_vespa = new_myd_vespa.dropna(subset=['IWV_MODIS','IWV_THAAO'])
new_modis_TOT_vespa = pd.concat([new_mod_vespa,new_myd_vespa], sort=True)

%store new_mod_vespa new_myd_vespa new_modis_TOT_vespa

In [ ]:
%store mod_data myd_data airs_data_asc airs_data_disc
%store cimel_data ftir_data hatpro_data radiosounding_data
%store vespa_data gnss_data